In [ ]:
# Hyper-paramètres

In [ ]:
# Les hyper-paramètres sont des valeurs fixées avant l'entraînement,
# influençant la performance, la stabilité et la vitesse de convergence
# du modèle. Contrairement aux paramètres appris, les hyper-paramètres
# nécessitent maîtrise et expérimentation.

In [ ]:
# Early stopping

In [ ]:
# L'early stopping consiste à arrêter l'entraînement quand les performances
# d'un modèle sur le jeu de validation cessent de s'améliorer. Il permet
# aussi d'éviter le surapprentissage.

# Avec PyTorch :
# Voir chapitre 3, exercice 2

In [ ]:
# Learning rate scheduler

In [ ]:
# Le learning rate influence la vitesse d'apprentissage d'un modèle.
# Le scheduler permet de modifier sa valeur selon une stratégie.
# Réduire le learning rate durant l'entraînement améliore souvent la 
# convergence et la performance du modèle.
# Plusieurs stratégies :
# - Réduction par palier
# - Réduction exponentielle
# - Basé sur la performance (patience)

# Avec PyTorch
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, ExponentialLR, CosineAnnealingLR, LinearLR
from torch.nn import Module, Linear, ReLU
from torch.optim import SGD

class MLP(Module):
  def __init__(self, features, neurons, classes):
    super(MLP, self).__init__()
    self.fc1 = Linear(features, neurons)
    self.fc2 = Linear(neurons, classes)
    self.relu = ReLU()

  def forward(self, x):
    x = self.relu(self.fc1(x))
    return self.fc2(x)

model = MLP()
optimizer = SGD(model.parameters(), lr=1e-2)

# Step scheduler, réduit à intervalles fixes 
step = StepLR(optimizer, step_size=10, gamma=0.1)

# ReduceLROnPlateau, réduction basée sur la patience
plateau_sch = ReduceLROnPlateau(optimizer, factor=0.5, patience=3)

# Exponential scheduler, réduction exponentielle
exp_sch = ExponentialLR(optimizer, gamma=0.95)

# Cos scheduler, réduction cosinusoïdale
cos_sch = CosineAnnealingLR(optimizer, T_max=50)

# Réduction linéaire
lin_sch = LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)

# Utilisation dans la boucle d'entraînement
for _ in range(50):
  # Toutes les étapes précédentes, et après optimizer.step():
  # Le nom du scheduler
  # scheduler.step(loss)
  pass

In [ ]:
# Momentum

In [ ]:
# Le moment (momentum), est une technique complémentaire aidant 
# à accélérer la convergence en accumulant une "vitesse" (inertie)
# dans la direction des gradients. Tous les optimisateurs basés sur 
# le gradient peuvent intégrer le momentum

In [ ]:
# Regularizer

In [ ]:
# Les régularisateurs pénalisent la complexité du modèle en ajoutant
# une contrainte aux poids. Cela permet d'éviter l'overfitting, en
# limitant la complexité du modèle à s'adapter trop étroitement aux
# données d'entrainement.

# Les régularisations les plus courantes sont :
# - L2 Regularisation (Ridge) : pénalise la somme des carrés des poids,
# encourageant des poids plus petits et répartis.
# - L1 Regularisation (Lasso) : pénalise la somme des valeurs absolues
# des poids, favorisant la sparité (beaucoup de poids = 0)
# - Dropout : éteint aléatoirement certains neurones (améliore la 
# généralisation).

# Régularisation L2
# L_total = L_original + λ·∑i -> w_i²
# où L_total est la nouvelle fonction de perte avec régularisation, 
# L_original la fonction de perte originale, λ le coefficient de
# régularisation, w_i représente les poids du modèle.

# Avec PyTorch
from torch.nn import MSELoss

optimizer = SGD(model.parameters(), lr=1e-2, weight_decay=1e-3)

# Regularisation L1
# L_total = L_original + λ·∑i -> |w_i|
# Cette régularisation favorise les poids = 0, ce qui peut être utile
# pour sélectionner automatiquement les caractéristiques les plus
# importantes dans les données.

# Pas d'implémentation avec PyTorch :
λ = 1e-3
criterion = MSELoss()

for _ in range(50):
  optimizer.zero_grad()
  loss = criterion(model(input), "target") # Target mis en commentaires
  # éviter les erreurs.

  # Ajout de la régularisation L1
  l1 = sum(p.abs().sum() for p in model.parameters())
  loss = loss + λ * l1
  
  loss.backward()
  optimizer.step()

# Dropout
# y_i {z_i avec probabilité 1-p et 0 avec probabilité p
# où y_i est la sortie du neurone, z_i est la sortie initiale du neurone
# et p la probabilité de dropout.
# Le Dropout est utile pour éviter la co-adaptation des poids ce qui 
# améliore la généralisation du modèle.

# Avec PyTorch :
# Dans la classe du MLP/CNN, etc, déclarer dans le __init__ :
# self.drop = Dropout(0.5), 0.5 représentant p.

In [ ]:
# Normalizers

In [ ]:
# Les réseaux de neurones sont sensibles aux distributions de données, à leur
# domaine de valeur. Il y a deux distributions de données :
# - In-distribution (ID), données que le modèle à l'habitude de voir et de traiter
# - Out-of-distribution (OOD), l'inverse du précédent.
# Le changement de distribution interne est appelé le internal covariate shift
# (décalage des covariables internes, ou ICS/DCI), et peut ralentir l'entraînement.

# Normalizers pour contrer le DCI
# ̂x = (x-μ(x)) ÷ (σ(x)+ε) × γ + β
# où ̂x est la valeur standardisée, x est la valeur d'entrée, μ(x) est la moyenne des
# activations, σ(x) est l'écart-type des activations, ε est une petite constante
# pour éviter la division par 0, γ et β sont des paramètres appris qui permettent
# de redimmentionner et de recentrer les activations standardisées.

# Il existe plusieurs normalizers, qui diffèrent dans la manière de calculer μ(x)
# et σ(x) ainsi que le moment où ils sont appliqués dans le réseau.
# Les plus courants :
# - Batch Normalization : standardise sur un mini-batch de données, aidant à stabiliser
# et accélérer l'entraînement.
# - Layer Normalisation : standardise sur toutes les caractéristiques d'une seule donnée,
# utile pour les architectures récurrentes.
# - Instance Normalization : standardise sur chaque canal d'une donnée. Utilisé en cv
# - Group Normalization : divise les canaux en groupes et standardise les activations
# au sein de chaque groupe. Utile lorsque la taille du batch est petite.

# Avec Pytorch
from torch.nn import BatchNorm2d, LayerNorm, InstanceNorm2d, GroupNorm

class ImageNormalizerNetwork(Module):
  def __init__(self):
    super(ImageNormalizerNetwork, self).__init__()
    self.batch = BatchNorm2d(3) # 3 canaux pour tout le batch
    self.layer = LayerNorm([3, 224, 224]) # chaque échantillon
    self.inst = InstanceNorm2d(3) # chaque canal
    self.group = GroupNorm(1, 3) # 1 groupe pour 3 canaux
  
  def forward(self, x):
    # Appeler les fonctions ci-dessus pour effectuer la standardisation.
    pass

In [ ]:
# Hyperparameter search/tuning

In [ ]:
# D'autres hyper-paramètres, différents que ceux que nous avons vu,
# sont importants, comme :
# - Le choix de l'optimisateur
# - Le learning rate initial
# - La taille du batch
# - L'architecture du modèle

# En Deep Learning, il nous faut trouver une combinaison optimale de 
# ces paramètres. La recherche d'hyper-paramètres est le processus
# d'optimisation de ces derniers, pour améliorer les performances
# d'un modèle. 
# Les stratégies les plus communes sont au nombre de 4 :
# - Grid Search : exploration exhaustive d'une grille prédéfinie d'hyper-
# paramètres, méthode simple mais coûteuse.
# - Random Search : souvent plus efficace que la recherche en grille, 
# surtout lorsque des hyper-paramètres ont plus d'impact que d'autres.
# - Bayezian Optimization : utilise des modèles probabilistes pour
# modéliser la fonction de performance, guide la recherche des paramètres
# vers les régions prometteuses de l'espace hyper-paramétrique.
# - Hyperband : combine recherche aléatoire avec early stopping pour
# allouer les ressources de calcul aux configurations les plus prometteuses.

# Quelques bibliothèques :
# - Optuna
# - Hyperopt
# - Ray Tune
# - Scikit-Optimize

In [ ]:
# Exercice 1